In [1]:
import sys

sys.path.append("/Users/shinewu/codes/xiang_wu_experimental/xiang-wu-experimental")

In [2]:
import collections
import csv
import itertools
import matplotlib.pyplot as plt
import json
import os
import pathlib
import re

import pandas as pd

from python import data, utils, word_matchers, word_models

In [3]:
print(word_matchers.extract_words("aa12 Bc! #Net$"))
print(word_matchers.split_at_capital("SuperCoolNFT"))
print(word_matchers.tokenize("moneyTake123 #BadApple$$ $N$"))

['aa', '12', 'Bc', 'Net', '$']
['Super', 'Cool', 'NFT']
['money', 'take', '123', 'bad', 'apple', '$$', 'n', '$']


In [4]:
MIN_30D_SALES = 1

ALL_COLLECTION_SQL = f"""
SELECT
  a.ID,
  a.NAME,
  a.DEFAULT_CHAIN_ID,
  a.SLUG,
  a.DESCRIPTION,
  b.THIRTY_DAY_SALES
FROM
  PC_FIVETRAN_DB.PUBLIC.API_COLLECTION a
  LEFT JOIN PC_FIVETRAN_DB.PUBLIC.API_COLLECTIONSTATS b
  ON a.ID = b.COLLECTION_ID
WHERE
  a.IS_DELISTED = FALSE
  AND a.HIDDEN = FALSE
  AND a.IS_NSFW = FALSE
  AND a.BLACKLISTED = FALSE
  AND b.THIRTY_DAY_SALES >= {MIN_30D_SALES}
ORDER BY THIRTY_DAY_SALES DESC
"""

print(ALL_COLLECTION_SQL)


SELECT
  a.ID,
  a.NAME,
  a.DEFAULT_CHAIN_ID,
  a.SLUG,
  a.DESCRIPTION,
  b.THIRTY_DAY_SALES
FROM
  PC_FIVETRAN_DB.PUBLIC.API_COLLECTION a
  LEFT JOIN PC_FIVETRAN_DB.PUBLIC.API_COLLECTIONSTATS b
  ON a.ID = b.COLLECTION_ID
WHERE
  a.IS_DELISTED = FALSE
  AND a.HIDDEN = FALSE
  AND a.IS_NSFW = FALSE
  AND a.BLACKLISTED = FALSE
  AND b.THIRTY_DAY_SALES >= 1
ORDER BY THIRTY_DAY_SALES DESC



In [5]:
RUN_SQL = False
ALL_COLLECTION_DF_FILE = pathlib.Path(f"/Users/shinewu/tmp/all_collection_min_30d_sales_{MIN_30D_SALES}.pickle")

if RUN_SQL:
    ALL_COLLECTION_DF = data.snowflake_run_query(ALL_COLLECTION_SQL)
    ALL_COLLECTION_DF.to_pickle(ALL_COLLECTION_DF_FILE)
else:
    ALL_COLLECTION_DF = pd.read_pickle(ALL_COLLECTION_DF_FILE)

In [6]:
ALL_COLLECTION_DF.iloc[:10]

,ID,NAME,DEFAULT_CHAIN_ID,SLUG,DESCRIPTION,THIRTY_DAY_SALES
0,8531189,Unidentified contract - 5lNnPz07oS,1,unidentified-contract-5lnnpz07os,None,1.000000e+22
1,8515777,FraktalNFTCyberBrokes V4,1,fraktalnftcyberbrokes-v4,None,1.000000e+22
2,13295152,Shattered Eon Raw Resources,1,shatteredeon-rawresources,"**Shattered Eon is an ever-expanding, ever-unf...",4.062781e+06
3,12922940,RicoWebmaster,1,eejiewieiwiuruiwrq,----,2.999900e+04
4,14198078,PoorLandNFT,1,poorlandnft,100 million Metaverse Land NFTs belonging to t...,2.902800e+04
5,14919425,AINightbirds,1,ainightbirds,"The stunning collection of 10,000 AINightbirds...",2.691000e+04
6,15130764,goblintown.wtf,1,goblintownwtf,AAAAAAAUUUUUGGGHHHHH gobblins goblinns GOBLINN...,2.561600e+04
7,15650945,Elftown.wtf,1,elftown-wtf,"The Dark Elves and High Elves, the two ancient...",2.443300e+04
8,14195734,Rare Land NFT,1,rarelandnft,Welcome to Rare Land.\nRare Land NFTs are the ...,2.333200e+04
9,15826563,ill poop it nft,1,pieceofshit-wtf,"yₒᵤᵣ ₙfₜ ᵢₛ ⱼᵤₛₜ ₐ ₚᵢₑcₑ ₒf ₛₕᵢₜ \n₁₀,₀₀₀ ₚᵢₗₑ...",2.069400e+04


In [16]:
def test_with(label, s):
    matcher = word_models.LABEL_AND_MATCHER[label]
    return list(word_matchers.match_all(matcher, word_matchers.tokenize(s)))

In [17]:
test_with(
    "GET_MONEY",
    "paying you no dollar and you get paid $$$ and you receive monthly payment"
)

[(6, 9)]

In [18]:
test_with("FINANCIAL_FREEDOM", "financial total freedom!")

[(0, 3)]

In [19]:
test_with("RETIRE_EARLY", "Retires pretty sure early!")

[(0, 4)]

In [20]:
test_with("SECURITY_TRADED", "stocks will be sold to him")

[(0, 4)]

In [11]:
def label_with_matcher(row):
    all_text = str(row["NAME"]) + " " + str(row["DESCRIPTION"])
    all_tokens = word_matchers.tokenize(all_text)


    labels = []
    patterns = []
    for label, matcher in word_models.LABEL_AND_MATCHER.items():
        matches = list(word_matchers.match_all(matcher, all_tokens))
        if matches:
            labels.append(label)
            patterns.append((all_tokens[slice(*matches[0])], len(matches)))
    return labels, patterns


LABELS_WITH_MATCHER = ALL_COLLECTION_DF.apply(label_with_matcher, axis=1)

In [21]:
desc = '''


Aiyah Verse Stickman-X Genesis 465 out of 666 Genesis Stickman-X have awakened on the Ethereum ERC721 blockchain. Nobody know when the rest will do so. These Stickman-X are the origins of the vast Aiyah Verse and will allow their masters to travel between the worlds within. New worlds are uncovered as time moves forward... to explore the new or to dominate the old, their fate in the Aiyah Verse is yours to decide.

Holders of the Stickman-X Genesis will be entitled to profit-sharing in our upcoming mobile game - Crypto Warframe as well as secondary sale revenue-sharing in our upcoming ID NFTs to be used in the game


'''

print(test_with("REVENUE_SHARE", desc))

[(87, 89), (101, 103)]


In [13]:
GROUPED = collections.defaultdict(list)

num_flagged = 0

for row, (label, matched) in zip(ALL_COLLECTION_DF.itertuples(), LABELS_WITH_MATCHER):
    if label:
        num_flagged += 1
    for l, (first_m, num) in zip(label, matched):
        GROUPED[l].append(dict(
            first_match=first_m,
            num=num,
            slug=row.SLUG,
            name='' if row.NAME is None else row.NAME,
            desc='' if row.DESCRIPTION is None else row.DESCRIPTION,
        ))

print(f"positive rate = {num_flagged} / {len(ALL_COLLECTION_DF)} = {num_flagged / len(ALL_COLLECTION_DF)}", num_flagged)

positive rate = 416 / 38857 = 0.010705921712947474 416


In [14]:
for label, rows in GROUPED.items():
    print(f"\n\n\n ===== {label} BEGIN ===== \n\n\n")
    for row in rows[:20]:
        print(f"{utils.slug_to_url(row['slug'])}  ,  label: {label}")
        print(f"num_matchs={row['num']}, first_match={row['first_match']}")
        print(row["name"])
        print(row["desc"][:100])
        print("----------\n")

    print(f"\n\n\n ===== {label} END ===== \n\n\n")




 ===== PASSIVE_INCOME BEGIN ===== 



https://opensea.io/collection/fantasyapeyachtclub  ,  label: PASSIVE_INCOME
num_matchs=1, first_match=['passive', 'income']
Fantasy Ape Yacht Club
"Fantasy Ape Yacht Club'' is a private collection of unique digital 50 NFTs stored as ERC-721 tokens
----------

https://opensea.io/collection/dopyapeyachtclub  ,  label: PASSIVE_INCOME
num_matchs=1, first_match=['passive', 'income']
Dopy Ape Yacht Club
The most visited place in the yacht club, where the monkeys will not be bored! "Dopy Ape Yacht Club'
----------

https://opensea.io/collection/pugging-awesome  ,  label: PASSIVE_INCOME
num_matchs=2, first_match=['diversified', 'investment']
Pugging Awesome
5555 Pixelated Pugs that grant holders access to the most diversified investment fund on the Solana 
----------

https://opensea.io/collection/degenerate-bear-syndicate-1  ,  label: PASSIVE_INCOME
num_matchs=1, first_match=['passive', 'income']
Degenerate Bear Syndicate
Degenerate Bear Syndicate is a

In [15]:
for label, rows in GROUPED.items():
    print(f"{label} -> {len(rows)}")

PASSIVE_INCOME -> 115
REVENUE_SHARE -> 43
INTEREST_GOT -> 16
GET_MONEY -> 60
SECURITY_UNIGRAM -> 38
GET_INTEREST -> 46
GET_PROFIT -> 64
IS_INVESTMENT -> 34
FINANCIAL_FREEDOM -> 9
MONEY_GOT -> 9
TRADE_SECURITY -> 7
PROFIT_GOT -> 11
INCREASE_INVESTMENT -> 5
RETIRE_EARLY -> 2
